PostId: 2019-04-30-222621
Title:DNNの情報理論的な論文を読む

Tishbyが良く研究しているっぽいので、その辺を中心に読んでいく。

## Deep Learning and the Information Bottleneck Principle

[arxiv1503.02406: Deep Learning and the Information Bottleneck Principle (2015)](https://arxiv.org/abs/1503.02406)

2015年。DNNの学習をIBの枠組みで見てみる。
入力と出力のMutual Information（以下MI）とか、hidden layerとhidden layerとhiddenの間のMIだとかを見ていく。
何かが分かったというよりは、IB的な枠組みで捉えなおすとこうなりますよね、という内容。

Cover and Thomasで出てくるRate distortion theoryとか十分統計量とかData proccessing inequalityとか、
関連する話題が多くてなかなか面白い。

## Opening the Black Box of Deep Neural Networks via Information

[arxiv1703.00810: Opening the Black Box of Deep Neural Networks via Information](https://arxiv.org/abs/1703.00810)

先ほどの2015年論文の続編。

### 対象としているデータ

3に具体的な相互情報量の計算方法が載っている。
30個の離散値に離散化して計算、とかなかなか面白い。

ただ教師データが何なのかがいまいち分からない。
12 binary inputsで、2Dのsphere(つまり円か？)の12 uniformly distributed pointsだと言っている。
Xはこの4096パターンがどうこう、と言っているが、良く分からない。
4096とは12 binaryという事で2の12乗っぽいが…

[ソースはあった](https://github.com/ravidziv/IDNNs)が、var_u.matという行列データを食わせているだけで、このデータの作り方は不明。
うーん。

Kazhdan 2003を参考にしているっぽいなぁ。
[Rotation Invariant Spherical Harmonic Representation of 3D Shape Descriptors (pdf)](https://www.cs.princeton.edu/~funk/sgp03.pdf) これか。
軽く読んだ。

3次元のモデルデータを、何らかのdescriptorにする事で検索とかしやすくする、という物。
回転不変なdescriptorにする為、いろいろやって64*64の二次元gird上の値にしている（Figure 4.)

ラベルデータとしてはViewpointのオブジェクトのラベルか。
これがKazhdan 2003らしい。

これをこの論文の3.1に書いてあるように変形している訳だ。
つまり、ラベルを、各グリッドが対象としているモデルのうち、平均よりスペクトルが大きいか？というのをラベルとしている。

…ん？どういう事だ？つまり適当な3Dモデル一つを選んで、4096通りの2値の教師データを作ったという事か？
めっちゃ少ないが大丈夫なのか。
そしてこのモデルに使ったデータが元のViewpointのどのオブジェクトかはどこかに書いておいてくれないと、このデータ作り直せないじゃないか。
再現実験でも同じ.matファイルを使いまわしているっぽいし、これは酷いなぁ。

ちなみに提供されてるスクリプトで再現できない、みたいな事を言っているブログがあった。
[Failure to replicate Schwartz-Ziv and Tishby](https://planspace.org/20180213-failure_to_replicate_schwartz-ziv_and_tishby/)

なお、さらに続きの実験している方はもうちょっとちゃんとした再現手順を書いている。
[github: Code for On the Information Bottleneck Theory of Deep Learning](https://github.com/artemyk/ibsgd)

ただ以上の理解を元にすれば、ようするに4096通りのXとそれに応じた0か1のYがある訳で、これらのテーブルデータから離散的な同時分布が分かる訳だ。
P(X, Y)が分かるのは理解した。

### Tの分布とはなんぞや？

問題はP(T, X)だな。TはXから一意に決まってしまうので、同時分布なんて無いように思えるが。（しいて言えばデルタ関数だが）

解説とコードを眺めていると、i番目のアクティベーションの値のそれぞれを別々の実現値として分布を考えているように見えるな。
それっておかしくないか？例えば3番目のレイヤーの一番上のニューロンと二番目のニューロンのactivationの値は別の変数で、
両者の関係はかなり謎だと思うのだが。
それを同じ確率変数の分布として計算した物にどういう意味があるんだろう？

うーん、意味は分からんがコードは自分の理解と同じ事をしているように見えるな。
これは結構何をしているか怪しいような？この論文は結構引用されたりバズったりしてた印象だが…

まあいい。あるレイヤーのそれぞれのアクティベーションの値をTiという確率変数のそれぞれの実現値とみなしてMIを計算している。

### 学習の二つのフェーズの動画

論文には二つの動画を見ろ、とリンクが貼ってある。
二つ目のリンクはデッドリンクだった（なんなのだろう？）。

- [The optimization process in the Information Plane - symmetric rule](https://www.youtube.com/watch?v=P1A1yNsxMjc)

これは確かに主張の通りの綺麗な動画が取れているな。
動画の説明では色はどこのレイヤーかを表している、と言っている。

### 同一レイヤーの個々のニューロンの解釈は無意味

3.5の最後に、個々のニューロンの相関は無くなり、個々のニューロンのweightの解釈は無意味となる、と言っている。
どの辺からこれが言えているかはよく分からないが、
これがXとTの相互情報量で個々のニューロンのactivationを同一の確率変数の分布の結果のように考えるjustificationとなっている気もする。
説得力は微妙だが、根拠は一応ありそう（先に言ってよ、という気もするが）。

### 3.7 Focker-Planck equationってなんぞ？

拡散方程式の一種になる、という事で、制約条件的なのがついた形の知られている確率過程の問題になるらしい。
エクセンダールを見直したが同じ式は見つからなかった。
ただブラウン運動と似ているので、そういう感じのものなのだろう。

式11は衝撃的な内容で、こんな凄い事が本当に分かるの？と思ってしまうが、
Focker-Planck equationを知らないので本当かどうかも分からない。仕方ない。

感覚的にはそれぞれの場所でMaximum Entropyを解く感じになるので、式11的にそれぞれのエントロピーが下がるから学習しなくてはいけない分布もより単純になりそうな気はする。

この辺が本当だったらDeep LearningでDeepにする効果が分かったと言えちゃうよなぁ。
その割には実験の再現データの作り方とか皆からもっと突っ込み入りそうな物だが。

### 3.8 理論的なboundとの一致（が良く分からない）

とりあえず理論的なtの分布が出るのはいい。
で、これを収束した時の各hiddden layerの分布と比較する、なら話は分かるのだが、
この理論値にはベータというパラメータがある。
モデルの複雑さと答えの正確さのトレードオフみたいなのを調整する値。

で、このベータを、各hidden layerに最も近くなるように決める、で、決めたら近かった、と言っている。
うーん、うん？みたいな分からなさがあるな。